# Logging
To keep track of what is happening in your code, you can use the provided logging module.
Below you can see an example of how to use it.

In [12]:
# logging settings
from qsprpred.logs.utils import enable_file_logger
logSettings = enable_file_logger(
    log_folder = '../../tutorial_output/',
    filename = 'TutorialLogging.log',
    debug = False,
    disable_existing_loggers = False
)

A utility function is provided to save your conda environment to a file. This is useful if you want to share your code with others, or if you want to recreate your environment at a later time.

In [13]:
from qsprpred.logs.utils import export_conda_environment

# save the environment to a yaml file
export_conda_environment("../../tutorial_output/TutorialLoggingEnvironment.yml")

Environment exported to ../../tutorial_output/TutorialLoggingEnvironment.yml successfully!


Below we run a standard data preparation and modelling workflow to show logging output.

In [14]:
# Standard data preparation
import os
from qsprpred.data.data import QSPRDataset
from qsprpred.data.utils.descriptorsets import FingerprintSet
from qsprpred.data.utils.descriptorcalculator import MoleculeDescriptorsCalculator
from qsprpred.data.utils.datasplitters import RandomSplit

os.makedirs("../../tutorial_output/data", exist_ok=True)

dataset = QSPRDataset.fromTableFile(
  	filename='../../tutorial_data/A2A_LIGANDS.tsv', 
  	store_dir="../../tutorial_output/data",
  	name="LoggingTutorialDataset",
  	target_props=[{"name": "pchembl_value_Mean", "task": "REGRESSION"}],
  	random_state=42
)

feature_calculator = MoleculeDescriptorsCalculator(
    desc_sets = [FingerprintSet(fingerprint_type="MorganFP", radius=3, nBits=2048)]
)

dataset.prepareDataset(
    split=RandomSplit(test_fraction=0.2, dataset=dataset),
    feature_calculators=[feature_calculator],
)

dataset.save()

In [15]:
# Standard model training
from qsprpred.models.sklearn import SklearnModel
from qsprpred.models.metrics import SklearnMetric
from sklearn.neighbors import KNeighborsRegressor
from qsprpred.models.assessment_methods import CrossValAssessor, TestSetAssessor

os.makedirs("../../tutorial_output/models", exist_ok=True)

model = SklearnModel(
    base_dir = '../../tutorial_output/models',
    data = dataset,
    alg = KNeighborsRegressor,
    name = 'TutorialLoggingModel'
)

score_func = SklearnMetric.getDefaultMetric(model.task)
CrossValAssessor(score_func)(model)
TestSetAssessor(score_func)(model)

model.fitAttached()

model.save()

'tutorial_output/models/TutorialLoggingModel/TutorialLoggingModel_meta.json'